## DNN Retry Mean + Minimum
Included the minimum and the mean, as well as took out the data from 2020-2021, 2021-2022.

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from statistics import mean
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
from statistics import mean
import math

FUTURE_PREDICTION = 4

def findmax(x):
    maximum = 0
    for number in x:
        maximum = max(x)
    return maximum

def findmean(x):
    m = 0
    m = mean(x)
    return m

def findmin(x):
    m = 0
    m = min(x)
    return m

def CreateMean(dataframe,title,city):

    dataframe = dataframe.iloc[522:3924:1]
    dataframe.rename(columns = {f'{city}' : f'{title}_Mean'}, inplace = True)
    dataframe = dataframe.reset_index(drop=True)
    for x in dataframe.index:
        if x % 7 == 6:
            list = []
            mean = 0
            for y in range(x-6,x+1,1):
                list.append(dataframe.iloc[y][f'{title}_Mean'])
            mean = findmean(list)
            dataframe.at[x,f'{title}_Mean'] = mean
    for x in dataframe.index:
        if x % 7 != 6:
            dataframe.drop(index= x, inplace=True)
    dataframe = dataframe.reset_index(drop=True)
    return dataframe

def CreateMax(dataframe,title,city):

    dataframe = dataframe.iloc[522:3924:1]
    dataframe.rename(columns = {f'{city}' : f'{title}_Max'}, inplace = True)
    dataframe = dataframe.reset_index(drop=True)
    for x in dataframe.index:
        if x % 7 == 6:
            list = []
            maximum = 0
            for y in range(x-6,x+1,1):
                list.append(dataframe.iloc[y][f'{title}_Max'])
            maximum = findmax(list)
            dataframe.at[x,f'{title}_Max'] = maximum
    for x in dataframe.index:
        if x % 7 != 6:
            dataframe.drop(index= x, inplace=True)
    dataframe = dataframe.reset_index(drop=True)
    return dataframe

def CreateMin(dataframe,title,city):

    dataframe = dataframe.iloc[522:3924:1]
    dataframe.rename(columns = {f'{city}' : f'{title}_Min'}, inplace = True)
    dataframe = dataframe.reset_index(drop=True)
    for x in dataframe.index:
        if x % 7 == 6:
            list = []
            minimum = 0
            for y in range(x-6,x+1,1):
                list.append(dataframe.iloc[y][f'{title}_Min'])
            minimum = findmin(list)
            dataframe.at[x,f'{title}_Min'] = minimum
    for x in dataframe.index:
        if x % 7 != 6:
            dataframe.drop(index= x, inplace=True)
    dataframe = dataframe.reset_index(drop=True)
    return dataframe
    
def DataPreprocess(city):
    virus_df = pd.read_csv('Data/virus.csv')
    virus_df['Date'] = pd.to_datetime(virus_df['Date'])
    virus_df['year'] = virus_df['Date'].apply(lambda date: date.year)
    virus_df['month'] = virus_df['Date'].apply(lambda date: date.month)
    virus_df_new = virus_df[['Date','month','year',f'{city}']]
    virus_df_new.rename(columns = {f'{city}' : f'Virus_Count_{city}'}, inplace = True)

    temp_df = pd.read_csv(f'Data/{city}_temp.csv')
    pressure_df = pd.read_csv(f'Data/{city}_pressure.csv')
    hours_df = pd.read_csv(f'Data/{city}_hours.csv')
    rainfall_df = pd.read_csv(f'Data/{city}_rainfall.csv')

    temp_df['Date'] = pd.to_datetime(temp_df['Date'])
    temp_df['year'] = temp_df['Date'].apply(lambda date: date.year)
    temp_df['month'] = temp_df['Date'].apply(lambda date: date.month)

    pressure_df['Date'] = pd.to_datetime(pressure_df['Date'])
    pressure_df['year'] = pressure_df['Date'].apply(lambda date: date.year)
    pressure_df['month'] = pressure_df['Date'].apply(lambda date: date.month)
    pressure_df = pressure_df.sort_values(by="Date")
    pressure_df = pressure_df.reset_index(drop=True)

    hours_df['Date'] = pd.to_datetime(hours_df['Date'])
    hours_df['year'] = hours_df['Date'].apply(lambda date: date.year)
    hours_df['month'] = hours_df['Date'].apply(lambda date: date.month)
    hours_df = hours_df.sort_values(by="Date")
    hours_df = hours_df.reset_index(drop=True)

    rainfall_df['Date'] = pd.to_datetime(rainfall_df['Date'])
    rainfall_df['year'] = rainfall_df['Date'].apply(lambda date: date.year)
    rainfall_df['month'] = rainfall_df['Date'].apply(lambda date: date.month)
    rainfall_df = rainfall_df.sort_values(by="Date")
    rainfall_df = rainfall_df.reset_index(drop=True)

    list = []
    for x in temp_df.index:
        t = temp_df.iloc[x][f'{city}']
        svp = 6.1078*10**(7.5*t/(t+237.3))
        e = pressure_df.iloc[x][f'{city}']
        h = e/svp*100
        list.append(h)
        
    humidity_df = pd.DataFrame(data=list)
    humidity_df.rename(columns = {0:f'{city}'}, inplace = True)

    mean_temp_df = CreateMean(temp_df, 'Temperature',f'{city}')
    mean_pressure_df = CreateMean(pressure_df, 'Pressure',f'{city}')
    mean_hours_df = CreateMean(hours_df, 'Hours',f'{city}')
    mean_rainfall_df = CreateMean(rainfall_df, 'Rainfall',f'{city}')
    mean_humidity_df = CreateMean(humidity_df, 'Humidity',f'{city}')
    
    min_temp_df = CreateMin(temp_df, 'Temperature',f'{city}')
    min_pressure_df = CreateMin(pressure_df, 'Pressure',f'{city}')
    min_hours_df = CreateMin(hours_df, 'Hours',f'{city}')
    min_rainfall_df = CreateMin(rainfall_df, 'Rainfall',f'{city}')
    min_humidity_df = CreateMin(humidity_df, 'Humidity',f'{city}')

    df = pd.concat([virus_df_new,mean_temp_df,mean_pressure_df,mean_hours_df,mean_rainfall_df,mean_humidity_df,
                   min_temp_df,min_pressure_df,min_hours_df,min_rainfall_df,min_humidity_df], axis=1)
    df = df.loc[:,~df.columns.duplicated()]

    df['VirusCRM'] = df['Virus_Count_Tokyo'].rolling(14, center = True).mean()

    for i in df.index:
        if i == 0:
            df.at[i,'Gradient'] = 1
        else:
            if df.iloc[i-1]['VirusCRM'] == 0:
                if df.at[i,'VirusCDM'] > 0:
                    df.at[i,'Gradient'] = 1
                else:
                    df.at[i,'Gradient'] = 0
            else:
                grd = df.iloc[i]['VirusCRM']/(df.iloc[i-1]['VirusCRM'])
                df.at[i,'Gradient'] = grd
    
    for i in df.index:
        if (df.iloc[i]['Gradient'] >= 1.1) and (df.iloc[i+1]['Gradient'] >= 1.1) and (df.iloc[i+2]['Gradient'] >= 1.1):
            df.at[i,'Spreading'] = 1
        else:
            df.at[i,'Spreading'] = 0
    df = df.astype({'Spreading':'int8'})

    df_virus_count = df[f'Virus_Count_{city}']
    df.drop(columns=['VirusCRM',f'Virus_Count_{city}','year','Gradient'], axis=1, inplace=True)
    df_labels = df[['Date']]
    FUTURE_PREDICTION = 4
    for i in range(FUTURE_PREDICTION):
        df_labels[f"T+{(i+1)*7}"] = df_labels["Date"] + pd.DateOffset(days = (i+1)*7)
    temp = df[["Date", "Spreading"]]
    for i in range(FUTURE_PREDICTION):
        target_column = f"T+{(i+1)*7}"
        df_labels = df_labels.merge(temp, 
                                how ="left",
                                left_on = (target_column),
                                right_on=("Date"),
                                suffixes = ("","_y")
        )
        df_labels.rename(columns={"Spreading":f"{target_column}_Spreading"},inplace=True)
    df_labels.drop(columns=["Date_y"],inplace=True)

    for i in range(FUTURE_PREDICTION):
        df_labels.drop(columns =[f"T+{(i+1)*7}"],inplace=True,axis=1 )

    df = df.merge(df_labels,
             how='left',
             on = ["Date"])
    df.dropna(inplace=True)

    X = df.drop(columns=['T+7_Spreading','T+14_Spreading','T+21_Spreading','T+28_Spreading'],axis=1)
    y1 = df['T+21_Spreading']
    y2 = df['T+7_Spreading']
    y3 = df['T+14_Spreading']
    y4 = df['T+28_Spreading']

    return X, y1, y2, y3, y4, df, df_virus_count

In [3]:
X, y1, y2, y3, y4, df, df_virus_count = DataPreprocess('Tokyo')

/Users/miyanishio/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/var/folders/k5/3yc13bs57f7637937yjlstth0000gn/T/ipykernel_86650/862544602.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labels[f"T+{(i+1)*7}"] = df_labels["Date"] + pd.DateOffset(days = (i+1)*7)


In [4]:
X

,Date,month,Temperature_Mean,Pressure_Mean,Hours_Mean,Rainfall_Mean,Humidity_Mean,Temperature_Min,Pressure_Min,Hours_Min,Rainfall_Min,Humidity_Min,Spreading
0,2012-09-10,9,28.085714,26.942857,6.928571,0.928571,71.007732,27.3,25.2,3.5,0.0,67.299614,0
1,2012-09-17,9,28.414286,26.200000,8.257143,1.714286,67.730487,27.9,22.8,4.6,0.0,59.974362,0
2,2012-09-24,9,24.685714,24.600000,3.614286,19.285714,78.780351,20.3,20.1,0.0,0.0,70.685242,0
3,2012-10-01,10,23.314286,20.128571,4.928571,2.928571,69.795654,20.4,15.4,0.0,0.0,61.171464,0
4,2012-10-08,10,21.942857,19.657143,3.928571,9.214286,74.175200,19.1,14.9,0.0,0.0,64.521610,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,2021-11-01,11,15.328571,12.642857,5.528571,4.785714,72.719746,13.6,9.1,0.0,0.0,52.345705,0
478,2021-11-08,11,16.085714,12.257143,7.271429,0.357143,66.710138,14.9,9.9,4.6,0.0,56.221277,0
479,2021-11-15,11,14.771429,10.428571,7.600000,6.785714,61.393552,13.1,8.2,0.0,0.0,49.152989,0
480,2021-11-22,11,13.142857,11.185714,4.957143,5.285714,74.012661,12.5,9.1,0.0,0.0,58.809630,0


In [5]:
X = X.drop(columns=['Date'])

In [6]:
X = X[0:400] #until 2020 05-04
y1 = y1[0:400]
y2 = y2[0:400]
y3 = y2[0:400]
y4 = y2[0:400]

In [7]:
X

,month,Temperature_Mean,Pressure_Mean,Hours_Mean,Rainfall_Mean,Humidity_Mean,Temperature_Min,Pressure_Min,Hours_Min,Rainfall_Min,Humidity_Min,Spreading
0,9,28.085714,26.942857,6.928571,0.928571,71.007732,27.3,25.2,3.5,0.0,67.299614,0
1,9,28.414286,26.200000,8.257143,1.714286,67.730487,27.9,22.8,4.6,0.0,59.974362,0
2,9,24.685714,24.600000,3.614286,19.285714,78.780351,20.3,20.1,0.0,0.0,70.685242,0
3,10,23.314286,20.128571,4.928571,2.928571,69.795654,20.4,15.4,0.0,0.0,61.171464,0
4,10,21.942857,19.657143,3.928571,9.214286,74.175200,19.1,14.9,0.0,0.0,64.521610,0
...,...,...,...,...,...,...,...,...,...,...,...,...
395,4,12.271429,9.014286,6.542857,4.428571,63.967539,9.8,6.0,0.0,0.0,42.500413,0
396,4,11.557143,8.471429,6.914286,19.928571,63.265454,8.0,6.2,0.0,0.0,46.609958,0
397,4,12.300000,9.728571,6.228571,15.285714,68.356621,9.7,6.6,0.0,0.0,48.799968,0
398,4,13.657143,9.857143,6.671429,2.000000,62.859690,11.9,7.6,1.4,0.0,50.877661,0


In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
scaler1 = StandardScaler()
X_train, X_test = X[0:280], X[280:400]
y1_train, y1_test = y1[0:280], y1[280:400]

In [9]:
scaled_X_train = scaler1.fit_transform(X_train)
scaled_X_test = scaler1.transform(X_test)

In [10]:
model = Sequential()

model.add(Dense(12,activation="sigmoid"))
model.add(Dense(6,activation="sigmoid"))
model.add(Dense(3,activation="sigmoid"))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

2022-11-14 17:02:35.162749: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
early_stop = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=50)

In [12]:
model.fit(scaled_X_train,y1_train,epochs=400,validation_data=(scaled_X_test,y1_test),callbacks=[early_stop])

Epoch 1/400
9/9 [==============================] - 3s 74ms/step - loss: 0.6205 - accuracy: 0.8357 - val_loss: 0.5945 - val_accuracy: 0.9000
Epoch 2/400
9/9 [==============================] - 0s 16ms/step - loss: 0.6052 - accuracy: 0.8357 - val_loss: 0.5792 - val_accuracy: 0.9000
Epoch 3/400
9/9 [==============================] - 0s 16ms/step - loss: 0.5935 - accuracy: 0.8357 - val_loss: 0.5659 - val_accuracy: 0.9000
Epoch 4/400
9/9 [==============================] - 0s 43ms/step - loss: 0.5833 - accuracy: 0.8357 - val_loss: 0.5541 - val_accuracy: 0.9000
Epoch 5/400
9/9 [==============================] - 0s 41ms/step - loss: 0.5739 - accuracy: 0.8357 - val_loss: 0.5427 - val_accuracy: 0.9000
Epoch 6/400
9/9 [==============================] - 0s 37ms/step - loss: 0.5650 - accuracy: 0.8357 - val_loss: 0.5316 - val_accuracy: 0.9000
Epoch 7/400
9/9 [==============================] - 0s 6ms/step - loss: 0.5564 - accuracy: 0.8357 - val_loss: 0.5209 - val_accuracy: 0.9000
Epoch 8/400
9/9 [====

9/9 [==============================] - 0s 7ms/step - loss: 0.4092 - accuracy: 0.8357 - val_loss: 0.3273 - val_accuracy: 0.9000
Epoch 60/400
9/9 [==============================] - 0s 8ms/step - loss: 0.4080 - accuracy: 0.8357 - val_loss: 0.3265 - val_accuracy: 0.9000
Epoch 61/400
9/9 [==============================] - 0s 12ms/step - loss: 0.4067 - accuracy: 0.8357 - val_loss: 0.3258 - val_accuracy: 0.9000
Epoch 62/400
9/9 [==============================] - 0s 7ms/step - loss: 0.4055 - accuracy: 0.8357 - val_loss: 0.3251 - val_accuracy: 0.9000
Epoch 63/400
9/9 [==============================] - 0s 8ms/step - loss: 0.4043 - accuracy: 0.8357 - val_loss: 0.3243 - val_accuracy: 0.9000
Epoch 64/400
9/9 [==============================] - 0s 7ms/step - loss: 0.4030 - accuracy: 0.8357 - val_loss: 0.3236 - val_accuracy: 0.9000
Epoch 65/400
9/9 [==============================] - 0s 9ms/step - loss: 0.4018 - accuracy: 0.8357 - val_loss: 0.3230 - val_accuracy: 0.9000
Epoch 66/400
9/9 [==============

9/9 [==============================] - 0s 9ms/step - loss: 0.3266 - accuracy: 0.8357 - val_loss: 0.2856 - val_accuracy: 0.9000
Epoch 118/400
9/9 [==============================] - 0s 12ms/step - loss: 0.3251 - accuracy: 0.8357 - val_loss: 0.2852 - val_accuracy: 0.9000
Epoch 119/400
9/9 [==============================] - 0s 14ms/step - loss: 0.3236 - accuracy: 0.8357 - val_loss: 0.2845 - val_accuracy: 0.9000
Epoch 120/400
9/9 [==============================] - 0s 7ms/step - loss: 0.3221 - accuracy: 0.8357 - val_loss: 0.2841 - val_accuracy: 0.9000
Epoch 121/400
9/9 [==============================] - 0s 11ms/step - loss: 0.3207 - accuracy: 0.8357 - val_loss: 0.2834 - val_accuracy: 0.9000
Epoch 122/400
9/9 [==============================] - 0s 11ms/step - loss: 0.3192 - accuracy: 0.8357 - val_loss: 0.2826 - val_accuracy: 0.9000
Epoch 123/400
9/9 [==============================] - 0s 8ms/step - loss: 0.3179 - accuracy: 0.8357 - val_loss: 0.2819 - val_accuracy: 0.9000
Epoch 124/400
9/9 [====

9/9 [==============================] - 0s 9ms/step - loss: 0.2651 - accuracy: 0.8357 - val_loss: 0.2594 - val_accuracy: 0.9000
Epoch 176/400
9/9 [==============================] - 0s 11ms/step - loss: 0.2642 - accuracy: 0.8357 - val_loss: 0.2588 - val_accuracy: 0.9000
Epoch 177/400
9/9 [==============================] - 0s 16ms/step - loss: 0.2638 - accuracy: 0.8357 - val_loss: 0.2579 - val_accuracy: 0.9000
Epoch 178/400
9/9 [==============================] - 0s 7ms/step - loss: 0.2629 - accuracy: 0.8357 - val_loss: 0.2574 - val_accuracy: 0.9000
Epoch 179/400
9/9 [==============================] - 0s 9ms/step - loss: 0.2622 - accuracy: 0.8357 - val_loss: 0.2572 - val_accuracy: 0.8917
Epoch 180/400
9/9 [==============================] - 0s 8ms/step - loss: 0.2615 - accuracy: 0.8321 - val_loss: 0.2567 - val_accuracy: 0.8750
Epoch 181/400
9/9 [==============================] - 0s 10ms/step - loss: 0.2609 - accuracy: 0.8429 - val_loss: 0.2562 - val_accuracy: 0.8750
Epoch 182/400
9/9 [=====

9/9 [==============================] - 0s 9ms/step - loss: 0.2317 - accuracy: 0.8929 - val_loss: 0.2226 - val_accuracy: 0.9000
Epoch 234/400
9/9 [==============================] - 0s 27ms/step - loss: 0.2312 - accuracy: 0.8929 - val_loss: 0.2216 - val_accuracy: 0.9000
Epoch 235/400
9/9 [==============================] - 0s 7ms/step - loss: 0.2307 - accuracy: 0.8929 - val_loss: 0.2206 - val_accuracy: 0.9000
Epoch 236/400
9/9 [==============================] - 0s 7ms/step - loss: 0.2303 - accuracy: 0.8929 - val_loss: 0.2202 - val_accuracy: 0.9000
Epoch 237/400
9/9 [==============================] - 0s 10ms/step - loss: 0.2303 - accuracy: 0.8929 - val_loss: 0.2196 - val_accuracy: 0.9000
Epoch 238/400
9/9 [==============================] - 0s 8ms/step - loss: 0.2295 - accuracy: 0.8929 - val_loss: 0.2193 - val_accuracy: 0.9000
Epoch 239/400
9/9 [==============================] - 0s 6ms/step - loss: 0.2292 - accuracy: 0.8929 - val_loss: 0.2188 - val_accuracy: 0.9000
Epoch 240/400
9/9 [======

9/9 [==============================] - 0s 6ms/step - loss: 0.2118 - accuracy: 0.9036 - val_loss: 0.1954 - val_accuracy: 0.9167
Epoch 292/400
9/9 [==============================] - 0s 6ms/step - loss: 0.2116 - accuracy: 0.9036 - val_loss: 0.1953 - val_accuracy: 0.9167
Epoch 293/400
9/9 [==============================] - 0s 5ms/step - loss: 0.2113 - accuracy: 0.9036 - val_loss: 0.1947 - val_accuracy: 0.9167
Epoch 294/400
9/9 [==============================] - 0s 6ms/step - loss: 0.2110 - accuracy: 0.9036 - val_loss: 0.1938 - val_accuracy: 0.9167
Epoch 295/400
9/9 [==============================] - 0s 7ms/step - loss: 0.2106 - accuracy: 0.9036 - val_loss: 0.1933 - val_accuracy: 0.9167
Epoch 296/400
9/9 [==============================] - 0s 10ms/step - loss: 0.2103 - accuracy: 0.9036 - val_loss: 0.1930 - val_accuracy: 0.9167
Epoch 297/400
9/9 [==============================] - 0s 8ms/step - loss: 0.2104 - accuracy: 0.9036 - val_loss: 0.1929 - val_accuracy: 0.9167
Epoch 298/400
9/9 [=======

9/9 [==============================] - 0s 18ms/step - loss: 0.1991 - accuracy: 0.9143 - val_loss: 0.1837 - val_accuracy: 0.9167
Epoch 350/400
9/9 [==============================] - 0s 12ms/step - loss: 0.1992 - accuracy: 0.9143 - val_loss: 0.1841 - val_accuracy: 0.9167
Epoch 351/400
9/9 [==============================] - 0s 14ms/step - loss: 0.1989 - accuracy: 0.9143 - val_loss: 0.1840 - val_accuracy: 0.9167
Epoch 352/400
9/9 [==============================] - 0s 16ms/step - loss: 0.1986 - accuracy: 0.9143 - val_loss: 0.1838 - val_accuracy: 0.9167
Epoch 353/400
9/9 [==============================] - 0s 13ms/step - loss: 0.1984 - accuracy: 0.9143 - val_loss: 0.1836 - val_accuracy: 0.9167
Epoch 354/400
9/9 [==============================] - 0s 8ms/step - loss: 0.1982 - accuracy: 0.9143 - val_loss: 0.1834 - val_accuracy: 0.9167
Epoch 355/400
9/9 [==============================] - 0s 14ms/step - loss: 0.1979 - accuracy: 0.9179 - val_loss: 0.1831 - val_accuracy: 0.9167
Epoch 356/400
9/9 [==

In [13]:
predictions = model.predict(scaled_X_test)

In [14]:
preds = []
for x in predictions:
    if x >= 0.5:
        preds.append(1)
    else:
        preds.append(0)

In [15]:
from sklearn.metrics import classification_report,confusion_matrix

In [16]:
print(classification_report(y1_test,preds))

              precision    recall  f1-score   support

         0.0       0.97      0.94      0.95       108
         1.0       0.56      0.75      0.64        12

    accuracy                           0.92       120
   macro avg       0.77      0.84      0.80       120
weighted avg       0.93      0.92      0.92       120

